In [1]:
import re
import string
import nltk
import regex as re 
import pandas as pd 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import PunktSentenceTokenizer, word_tokenize
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
file_path = r"C:\Users\Sujal Karmakar\Desktop\Desktop\Data Analyst\Python\python_data_analytics_project\Theme Finder Using Caption (NLP)\Data\leanbeast_analysis_ready.csv"

df = pd.read_csv(file_path)

#### 1) Basic Cleaning (lower case, remover numbers, emoji, etc)

In [3]:
df_caption = df["caption_text"]

In [4]:
import re

# Your custom list of generic words and hashtags to remove (lowercase, no # for hashtags)
generic_words = set([
    "fitness", "gym", "workout", "fitnessmotivation", "bodybuilding", "training",
    "fit", "muscle", "fitfam", "gymlife", "shredded", "sixpack",
    "fitspo", "personaltrainer", "fitnessmodel", "exercise",
    "comment", "follow", "share", "message", "dm", "whatsapp", "send",
    "will", "i",
    "interested in paid transformation", "now", 
    "amazing", "total", "this is just an act",
    "free diet plan", "zero", "link", "tap", "check", "list"
])

# Optional: split multi-word phrases into words, or just handle them later as whole phrases.

def clean_caption(text):
    if pd.isna(text):
        return ""
    text = str(text).strip()
    # Lowercase the text for uniformity
    text_lower = text.lower()
    
    # Remove hashtags (#) before splitting, or treat hashtags specially
    # Here, replace hashtags with words without #
    text_lower = re.sub(r'#(\w+)', r'\1', text_lower)
    
    # Remove multiple spaces/newlines
    text_lower = re.sub(r'\s+', ' ', text_lower)
    
    # Remove the generic words from the text
    # To handle multi-word phrases, we can do iterative replacement:
    for phrase in generic_words:
        # Use regex to replace whole words/phrases only (case insensitive)
        # Add word boundaries \b to avoid partial matches
        pattern = r'\b' + re.escape(phrase) + r'\b'
        text_lower = re.sub(pattern, '', text_lower, flags=re.IGNORECASE)
    
    # Clean extra spaces again after removals
    text_lower = re.sub(r'\s+', ' ', text_lower).strip()
    
    return text_lower

# Apply to your caption column
df_caption_cleaned = df_caption.apply(clean_caption)

In [5]:
df_caption_cleaned.sample(15)

1165    how kaju is made …a good source of healthy fat...
855                                             oye mote😂
130     “diet” and you 100 gm veg protein diet in your...
730                                         fasting is on
775       papa ne bola bache nahi whey protein pene se😂😂…
1291    1 lakh calorie is stored in ur body in the for...
54                                15 whole eggs boiled❤️🫵
1006    he is one my fav actor after srk… started afte...
753         9km longest tunnel in the hills … atal tunnel
657                 tuff hai naturally tall guys ke liye😢
1136    ye sasta steroid se heart attack aa sakta hai ...
6       new jeep. new roads. endless adventures begin ...
1419    train with me show wat your body is capable of...
769                                          am the best…
286         “eggs” and you more such recipe in your inbox
Name: caption_text, dtype: object

#### 2) Tokenization (removing stop words)

In [6]:
# import nltk

# # Define a custom download directory
# custom_nltk_path = r'C:\Users\Sujal Karmakar\.conda\envs\DS\nltk_data'

# # Download resources to the custom path
## nltk.download('punkt', download_dir=custom_nltk_path)
# nltk.download('punkt_tab', download_dir=custom_nltk_path)
# nltk.download('stopwords', download_dir=custom_nltk_path)

# # Manually add the path to NLTK's search locations
# nltk.data.path.append(custom_nltk_path)

# i had to explicitly download these by specifying files and also punkt was not working for me nltk was demanding punkt_tab so i did that 


#### 3) Creating dictionaries and corpus for LDA (Linear discriminant analysis)

In [17]:
from bertopic import BERTopic

# Make sure df_caption_cleaned is a list of strings (captions)
documents = df_caption_cleaned.tolist()

# Option 1: Directly set nr_topics=12 when initializing the model
topic_model = BERTopic(language="english", nr_topics=25, verbose=True)

# Fit the model on your cleaned caption texts
topics, probs = topic_model.fit_transform(documents)

# Show the topic info (topic number, count, name)
topic_info = topic_model.get_topic_info()
print(topic_info)


# Show the top words for each topic except -1 (noise)
for topic_num in topic_info['Topic']:
    if topic_num != -1:  # -1 is the noise topic
        print(f"\nTopic {topic_num}:")
        print(topic_model.get_topic(topic_num))


2025-06-01 15:13:22,436 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/49 [00:00<?, ?it/s]

2025-06-01 15:13:33,279 - BERTopic - Embedding - Completed ✓
2025-06-01 15:13:33,280 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-06-01 15:13:35,700 - BERTopic - Dimensionality - Completed ✓
2025-06-01 15:13:35,701 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-06-01 15:13:35,749 - BERTopic - Cluster - Completed ✓
2025-06-01 15:13:35,749 - BERTopic - Representation - Extracting topics using c-TF-IDF for topic reduction.
2025-06-01 15:13:35,823 - BERTopic - Representation - Completed ✓
2025-06-01 15:13:35,824 - BERTopic - Topic reduction - Reducing number of topics
2025-06-01 15:13:35,836 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-06-01 15:13:35,885 - BERTopic - Representation - Completed ✓
2025-06-01 15:13:35,888 - BERTopic - Topic reduction - Reduced number of topics from 28 to 25


    Topic  Count                                       Name  \
0      -1    362                           -1_the_and_of_to   
1       0    256                  0_biceps_abs_cardio_gains   
2       1    170                          1_hai_nahi_meh_ka   
3       2     77                    2_you_inbox_recipe_eggs   
4       3     74          3_transformation_paid_lost_months   
5       4     70                 4_protein_veg_source_grams   
6       5     63  5_wedding_photography_happy_photooftheday   
7       6     55                  6_natty_mykonos_wife_itna   
8       7     54          7_india_iamleanbeastindia_he_with   
9       8     54     8_healthylifestyle_sport_crossfit_love   
10      9     48                        9_sugar_is_to_juice   
11     10     38                  10_fatloss_to_meal_weight   
12     11     32                    11_ice_icebath_bath_the   
13     12     26           12_fasting_hours_autophagy_start   
14     13     25              13_cholesterol_high_the_h

In [18]:
with open("bertopic_topics.txt", "w", encoding="utf-8") as f:
    for idx, topic in topic_model.get_topic_info().iterrows():
        f.write(f"Topic {topic['Topic']} (Count: {topic['Count']}):\n")
        terms = topic_model.get_topic(topic['Topic'])
        for term, weight in terms:
            f.write(f"  {term}: {weight:.4f}\n")
        f.write("\n")

In [19]:
# Mapping topic IDs to clear, professional names
topic_labels = {
    -1: "Nutrition & Diet Tips",
    0: "Fitness & Exercise",
    1: "Health Awareness",
    2: "Nutrition & Diet Tips",
    3: "Weight Management",
    4: "Nutrition & Diet Tips",
    5: "Lifestyle & Motivation",
    6: "Insights & Commentary ",
    7: "Lifestyle & Motivation",
    8: "Fitness & Exercise",
    9: "Sugar & Metabolic Health",
    10: "Weight Management",
    11: "Recovery & Injury Prevention",
    12: "Health Awareness",
    13: "Health Awareness",
    14: "Lifestyle & Motivation",
    15: "Nutrition & Diet Tips",
    16: "Health Awareness",
    17: "Health Awareness",
    18: "Nutrition & Diet Tips",
    19: "Health Awareness",
    20: "Lifestyle & Motivation",
    21: "Nutrition & Diet Tips",
    22: "Nutrition & Diet Tips",
    23: "Sugar & Metabolic Health",
}

# Add the human-readable topic name to the dataframe
df_result = pd.DataFrame({"Caption": df_caption, "Topic_ID": topics})
df_result["Topic_Name"] = df_result["Topic_ID"].map(topic_labels)

In [20]:
# Number of samples per topic you want to see
n_samples = 5

# Loop through each topic and print n samples
for topic in df_result["Topic_Name"].unique():
    print(f"\n🔷 Topic: {topic}")
    samples = df_result[df_result["Topic_Name"] == topic]["Caption"].sample(n_samples)
    for i, caption in enumerate(samples, 1):
        print(f"  {i}. {caption}")



🔷 Topic: Weight Management
  1. Sweating is not a fatloss guys just beware
  2. Well done @deepali50666 6 weeks weight loss changes plus inch loss   ✅If you wish to transform (paid training) whatsapp me now +91 9171123324 ✅
  3. Lost 18kgs in 4 months @dakshrajjajoria   DM ME FOR PAID TRANSFORMATION
  4. Comment “Low Sugar” and i will send you 5 drinks which will help you in fatloss and fasting.
  5. Comment “Diet” and i will send you a sample veg fatloss diet for beginners.

🔷 Topic: Health Awareness
  1. Accident hona bhi life ka hissa hai  I have injured my right yesterday in an accident, many of u commenting its showing left here ?? It is a selfie video of reels check tshirt brand name closely u will figure out🤝
  2. THIS VIDEO IS MEANT FOR FUN PURPOSE…  PAYTM KARO JALDI😂😂😂
  3. Sab cheeze promote nahi kar sakta meh🙏
  4. Kitne hours aur kitne din workout kare week meh??  Minimum itna time aap do
  5. Aaj ghar par workout karengay..

🔷 Topic: Nutrition & Diet Tips
  1. Fondue is a

In [11]:
df_result.to_csv("caption_topics_labeled.csv", index=False)
